# CEO-REQUEST CHALLENGE

> Should Olist remove underperforming sellers from its marketplace?

## Problem statement

To analyse the impact of removing the worse sellers from the marketplace, we can start with a what-if analysis: what would have happened if Olist had never accepted these sellers in the past? For that:

Step ① Compute, for each `seller_id`, cumulated since the beginning:
- The `revenues` they bring
- The `costs` associated with all their bad reviews
- The resulting `profits` (revenues - costs)
- The number of `orders` (it will impact the overall IT costs)


Step ② We can then sort sellers by increasing profits for Olist, and for different numbers of sellers removed, compute the financial impact it would have made had they never been accepted on the platform. We may find an optimal number of sellers to remove that maximizes Olist's profit margin

In [1]:
# Write down a detailed strategy (step by step) to create the DataFrame you need for step (1)
# Think about how to re-use logic you have already coded in your `olist` package

<details>
    <summary>Hints</summary>


Starting from your current `seller().get_training_data()` DataFrame:
- Can you easily transform it to compute Olist's positive `revenue_per_seller`? 
- Can you easily transform it to compute Olist's `cost_of_bad_reviews`?

❓Instead of starting again from scratch, investigate in your source code in `seller.py` how the mean `review_score` per seller was computed. Can you imagine a way to ammend your code so as to compute `cost_of_reviews` in the same process? 
</details>


## Your turn!

In [2]:
# Keep this notebook tidy, you will present it to the Olist CEO on Friday afternoon!

# Solution (remove from data-challenge)

Suggested strategy:
- Update `seller.py` to compute a missing `cost_of_reviews` column
- Compute `months_on_olist` and `revenues` using existing seller_training_data columns
- Compute `profits` per seller
- Sort DataFrame per `profits`
- Remove worst performing sellers one by one, and for each scenario:
    - compute `n_orders` and therefore `olist_it_costs` (proportional to square root of n_orders)
    - compute Olist's total `net_profits` and `margin`
- Find the optimal number of sellers to remove from the platform so as to maximize `profit` or `margin`

## Step ① : Update `seller.py`

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [4]:
# seller.py has been updated to seller_updated.py to automatically compute what we need for step (1)
from olist.seller_updated import Seller
seller = Seller()

In [9]:
sellers = seller.get_training_data()
sellers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2967 entries, 0 to 2966
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   seller_id            2967 non-null   object        
 1   seller_city          2967 non-null   object        
 2   seller_state         2967 non-null   object        
 3   delay_to_carrier     2967 non-null   float64       
 4   wait_time            2967 non-null   float64       
 5   date_first_sale      2967 non-null   datetime64[ns]
 6   date_last_sale       2967 non-null   datetime64[ns]
 7   share_of_one_stars   2967 non-null   float64       
 8   share_of_five_stars  2967 non-null   float64       
 9   review_score         2967 non-null   float64       
 10  n_orders             2967 non-null   int64         
 11  quantity             2967 non-null   int64         
 12  quantity_per_order   2967 non-null   float64       
 13  sales                2967 non-nul

In [13]:
sellers['subscription'] = np.floor((sellers.date_last_sale - sellers.date_first_sale).dt.days / 365 * 12 + 1) * 80

In [14]:
sellers

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,share_of_one_stars,share_of_five_stars,review_score,n_orders,quantity,quantity_per_order,sales,subscription
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,0.333333,0.333333,3.00,3,3,1.000000,218.70,320.0
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,0.050000,0.725000,4.55,40,41,1.025000,11703.07,1200.0
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.000000,1.000000,5.00,1,1,1.000000,158.00,80.0
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.000000,1.000000,5.00,1,1,1.000000,79.99,80.0
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,1.000000,0.000000,1.00,1,1,1.000000,167.99,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,98dddbc4601dd4443ca174359b237166,sarandi,PR,0.000000,8.230822,2018-07-14 03:10:19,2018-07-23 15:34:03,0.000000,1.000000,5.00,2,2,1.000000,158.00,80.0
2963,f8201cab383e484733266d1906e2fdfa,palhoca,SC,0.000000,13.045797,2017-06-17 15:30:11,2018-08-11 19:55:17,0.100000,0.500000,4.10,10,10,1.000000,889.00,1120.0
2964,74871d19219c7d518d0090283e03c137,sao paulo,SP,0.000000,3.606625,2018-04-05 17:09:46,2018-08-21 19:25:15,0.000000,1.000000,5.00,6,7,1.166667,550.04,400.0
2965,e603cf3fec55f8697c9059638d6c8eb5,pelotas,RS,0.000000,6.657634,2017-02-09 02:43:17,2017-06-16 17:30:16,0.000000,0.500000,4.25,8,11,1.375000,297.00,400.0


In [15]:
sellers['commission'] = sellers.sales * 0.1
sellers.head(3)

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,share_of_one_stars,share_of_five_stars,review_score,n_orders,quantity,quantity_per_order,sales,subscription,commission
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.0,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,0.333333,0.333333,3.00,3,3,1.000,218.70,320.0,21.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.0,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,0.050000,0.725000,4.55,40,41,1.025,11703.07,1200.0,1170.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.0,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.000000,1.000000,5.00,1,1,1.000,158.00,80.0,15.800


In [18]:
sellers['revenues'] = sellers['subscription'] + sellers['commission']

## Step ② : What-if analysis

### Olist P&L (big picture)

_Revenues_

In [21]:
sellers.drop(columns='revenue', inplace=True)

In [22]:
# Gross Revenues
olist_revenues = sellers.revenues.sum()
olist_revenues

2926538.444

In [23]:
# Including from monthly subscription
olist_subscription_revenues = sellers.months_on_olist.sum()*80
olist_subscription_revenues

AttributeError: 'DataFrame' object has no attribute 'months_on_olist'

In [24]:
# Including from sales
olist_sales_revenues = sellers.sales.sum()*0.1
olist_sales_revenues

1355578.4440000001

_Costs_

In [26]:
sellers

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,share_of_one_stars,share_of_five_stars,review_score,n_orders,quantity,quantity_per_order,sales,subscription,commission,revenues
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,0.333333,0.333333,3.00,3,3,1.000000,218.70,320.0,21.870,341.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,0.050000,0.725000,4.55,40,41,1.025000,11703.07,1200.0,1170.307,2370.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.000000,1.000000,5.00,1,1,1.000000,158.00,80.0,15.800,95.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.000000,1.000000,5.00,1,1,1.000000,79.99,80.0,7.999,87.999
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,1.000000,0.000000,1.00,1,1,1.000000,167.99,80.0,16.799,96.799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,98dddbc4601dd4443ca174359b237166,sarandi,PR,0.000000,8.230822,2018-07-14 03:10:19,2018-07-23 15:34:03,0.000000,1.000000,5.00,2,2,1.000000,158.00,80.0,15.800,95.800
2963,f8201cab383e484733266d1906e2fdfa,palhoca,SC,0.000000,13.045797,2017-06-17 15:30:11,2018-08-11 19:55:17,0.100000,0.500000,4.10,10,10,1.000000,889.00,1120.0,88.900,1208.900
2964,74871d19219c7d518d0090283e03c137,sao paulo,SP,0.000000,3.606625,2018-04-05 17:09:46,2018-08-21 19:25:15,0.000000,1.000000,5.00,6,7,1.166667,550.04,400.0,55.004,455.004
2965,e603cf3fec55f8697c9059638d6c8eb5,pelotas,RS,0.000000,6.657634,2017-02-09 02:43:17,2017-06-16 17:30:16,0.000000,0.500000,4.25,8,11,1.375000,297.00,400.0,29.700,429.700


In [25]:
olist_reputation_costs = sellers.cost_of_reviews.sum()
olist_reputation_costs

AttributeError: 'DataFrame' object has no attribute 'cost_of_reviews'

In [ ]:
olist_it_costs_all_orders = 500000

In [ ]:
# Total costs
olist_total_costs = 500000 + sellers.cost_of_reviews.sum()
olist_total_costs

_Profits_

In [ ]:
# before IT costs
olist_gross_profits = sellers.profits.sum()
olist_gross_profits

In [ ]:
# after IT costs
olist_net_profits = olist_gross_profits - olist_it_costs_all_orders
olist_net_profits

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Waterfall(
    orientation = "v",
    measure = ["relative", "relative", "total", "relative", "relative", "total"],
    x = ["Monthly subcriptions", "Sales", "Total Revenues", "Reputation costs", "It costs", "Net Profit"],
    textposition = "outside",
#    text = ["", "+80", "", "-40", "-20", "Total"],
    y = [olist_subscription_revenues, olist_sales_revenues, 0, -olist_reputation_costs, -olist_it_costs_all_orders, 0],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = "Olist P&L since inception (Million BRL)",
        showlegend = True
)

fig.show()

### Seller cut-off analysis

In [27]:
sns.histplot(sellers.profits, kde=False)

AttributeError: 'DataFrame' object has no attribute 'profits'

In [29]:
sorted_sellers = sellers.sort_values(by='profits')[['profits', 'n_orders', 'revenues']].reset_index()
sorted_sellers.head()

KeyError: 'profits'

In [ ]:
sorted_sellers.profits.plot();

#### Compute Olist's net_profits for various seller cut-offs

Let's create arrays `xxx_per_seller_removed` which list the `xxx` value for each number of sellers removed (from 1 to 3000 sellers)


In [28]:
# Analysis excluding IT costs
revenues_per_seller_removed = olist_revenues - np.cumsum(sorted_sellers.revenues)
gross_profits_per_seller_removed = olist_gross_profits - np.cumsum(sorted_sellers.profits)

gross_profits_per_seller_removed.plot()
plt.title('Olist gross profit before IT costs')
plt.xlabel('number of seller to remove')

NameError: name 'sorted_sellers' is not defined

In [ ]:
# Add the IT costs of Olist's platform
# IT costs =  A * (n_orders)**0.5

# First, compute A
A = olist_it_costs_all_orders / (sellers['n_orders'].sum()**0.5)
A

In [ ]:
# Then, compute n_order per scenario
n_orders_per_seller_removed = sorted_sellers.n_orders.sum() - np.cumsum(sorted_sellers.n_orders)
plt.plot(n_orders_per_seller_removed)
plt.title('Total number of orders'); plt.xlabel('number of seller to remove');

In [ ]:
# Create array of it_costs per scenario
it_costs_per_seller_removed = A * n_orders_per_seller_removed**0.5
plt.plot(it_costs_per_seller_removed)
plt.title('IT costs'); plt.xlabel('number of seller to remove');

In [ ]:
# Compute final metrics (profits and margin, for each scenario!)
profits_per_seller_removed = gross_profits_per_seller_removed - it_costs_per_seller_removed
margin_per_seller_removed = profits_per_seller_removed / revenues_per_seller_removed

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,5))
x = np.arange(0, len(sellers), 1)

plt.suptitle("Estimated impact of having excluding selected sellers from Olist, over the past 16 months")

ax1.set_xlabel('number of seller to remove from olist - from worse to most profitable')
ax1.set_ylabel('BRL')
ax1.plot(x, profits_per_seller_removed, color='black', label='Olist net profits')
ax1.plot(x, revenues_per_seller_removed, color='grey', label='Olist total revenues')
ax1.legend()
ax1.tick_params(axis='y')
ax1.legend()
ax1.set(ylim=[0,3000000])


ax3 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax3.set_ylabel('%', color=color)  # we already handled the x-label with ax1
ax3.plot(x, margin_per_seller_removed, color='red', label='Olist profit margin')
ax3.tick_params(axis='y', labelcolor=color)
ax3.legend(loc='right')
ax3.set(ylim=[0,1])

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

☝️We can clearly see a trade-off with a maximim net_profit at around 500 sellers removed, and a maximum margin at about 2000 sellers removed. Beyond this, the margin collapses due to IT costs.

🔎 Going further, we should improve this analysis by taking into account the dynamics: Olist does not know in advance a seller's performance. Sellers should only be removed from the platform after they have been receiving bad reviews for a given period.